In [1]:
import os
from pyltp import SentenceSplitter
################ 分词：################
LTP_DATA_DIR = 'F:/MyDownloads/ltp_data_v3.4.0/ltp_data_v3.4.0'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`

from pyltp import Segmentor
segmentor = Segmentor()  # 初始化实例
segmentor.load(cws_model_path)  # 加载模型
words = segmentor.segment('元芳你怎么看')  # 分词
print(list(words))
# segmentor.release()  # 释放模型

['元芳', '你', '怎么', '看']


In [2]:
################ 词性标注：################
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`

from pyltp import Postagger
postagger = Postagger() # 初始化实例
postagger.load(pos_model_path)  # 加载模型

words = ['元芳', '你', '怎么', '看']  # 分词结果
postags = postagger.postag(words)  # 词性标注
print(list(postags))
# postagger.release()  # 释放模型

['nh', 'r', 'r', 'v']


In [9]:
################ 依存句法分析：################
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`

from pyltp import Parser
parser = Parser() # 初始化实例
parser.load(par_model_path)  # 加载模型

relation_dic = {'SBV':'主谓关系','VOB':'动宾关系','IOB':'间宾关系','FOB':'前置宾语',\
                'DBL':'兼语','ATT':'定中关系','ADV':'状中结构','CMP':'动补结构',\
                'COO':'并列关系','POB':'介宾关系','LAD':'左附加关系','RAD':'右附加关系','IS':'独立结构','HED':'核心关系','WP':'标点'}

my_sentence = '档案的归纳、整理'

def get_words_and_tags(sentence):
    words = list(segmentor.segment(sentence))
    postags = list(postagger.postag(words))
    return [words,postags]

words,postags = get_words_and_tags(my_sentence)
arcs = parser.parse(words, postags)  # 句法分析

for index,arc in enumerate(arcs):
    relation_word = words[arc.head-1] if arc.head>0 else 'ROOT'
    relation = arc.relation
    combination = words[index]+relation_word if index<=arc.head-1 else relation_word+words[index]
    
    print('('+str(index)+') '+words[index],'|',\
            '('+str(arc.head-1)+') '+relation_word,'|',\
            relation,'|',relation_dic[relation],'|',combination)

#     if relation in ['SBV','VOB','ATT','COO']:
#         combination = words[index]+relation_word if index<=arc.head-1 else relation_word+words[index]
#         print('('+str(index)+') '+words[index],'|',\
#               '('+str(arc.head-1)+') '+relation_word,'|',\
#               relation,'|',relation_dic[relation],'|',combination)

# print("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs))
parser.release()  # 释放模型

(0) 档案 | (2) 归纳 | ATT | 定中关系 | 档案归纳
(1) 的 | (0) 档案 | RAD | 右附加关系 | 档案的
(2) 归纳 | (-1) ROOT | HED | 核心关系 | ROOT归纳
(3) 、 | (2) 归纳 | WP | 标点 | 归纳、
(4) 整理 | (2) 归纳 | COO | 并列关系 | 归纳整理


In [4]:
for i,each in enumerate(arcs):
    if each.relation == 'VOB':
        print(words[each.head-1],"--",words[i])

In [5]:
import re
def beyondSplit(x):
    sentences = re.split('。|，|？|！|：|；|\.|,|\?|\!|:|;',x)
    return sentences
beyondSplit("阿斯顿。发爱，迪生：念;佛；我:爱？上你！都该.但是,那?个阿斯!蒂芬")

['阿斯顿', '发爱', '迪生', '念', '佛', '我', '爱', '上你', '都该', '但是', '那', '个阿斯', '蒂芬']